Recursive formula

Given string $s$ and a scoring function $T$ that decides if two letters are matches, we get the following recurrence:
$$T(a,b) = \begin{cases}
1 & \text{if (a,b) can form a base pair}\\
0,& \text{otherwise}
\end{cases}$$

$$maxPairs(i,j) = max\begin{cases}
0 & \text{if $i >= j$}\\
maxPairs(i+1,j) \\
maxPairs(i,j-1) \\
1 + maxPairs(i+1,j-1) & \text{if $s_i = s_j$}\\
argmax\text{ } \underset{i < k < j} k maxPairs(i,k) + maxPairs(k+1,j)
\end{cases}$$

The final case in the recurrence handles bifurcation. Conceptually, it breaks the string into two parts (one starting with $i$ and ending with $k$, and the other starting with $k+1$ and ending with $j$), and solves each part independently.

In [1]:
T = {
    'A': 'U',
    'G': 'C',
    'C': 'G',
    'U': 'A',
}

In [2]:
# no DP used here, just recursion
def nussinov_recursive(s, i, j, T):
    if i >= j:
        return 0
    
    bifurcation = max([nussinov_recursive(s,i,k,T) + nussinov_recursive(s,k+1,j,T) for k in range(i+1,j)]) if j-i >= 2 else 0
    is_match = 1 + nussinov_recursive(s,i+1,j-1,T) if T[s[i]] == s[j] else -float('inf')
    
    return max(nussinov_recursive(s,i+1,j,T), nussinov_recursive(s,i,j-1,T), is_match, bifurcation)

In [3]:
s = 'ACGU'

In [4]:
nussinov_recursive(s, 0, len(s)-1, T)

2

In [5]:
s = 'AAAAU'

In [6]:
nussinov_recursive(s, 0, len(s)-1, T)

1

In [7]:
s = 'AAAAUUGC'

In [8]:
nussinov_recursive(s, 0, len(s)-1, T)

3

In [9]:
T = {
    'A': 'U',
    'G': 'C',
    'C': 'G',
    'U': 'A',
}

In [10]:
s = 'GGGAAAUCC' # should be 3

In [11]:
nussinov_recursive(s, 0, len(s)-1, T)

3

Now time to use dynamic programming

In [12]:
import numpy as np

In [13]:
def nussinov(s, T, return_table=False):
    """
    DP solution to nussinov
    """
    
    table = np.zeros((len(s), len(s)))
    
    # we fill the table diagonally
    cur_i, cur_j = 0, 1
    next_i_start, next_j_start = 0, 2
    
    while cur_i != 0 or cur_j != len(s):
        if cur_j - cur_i >= 2: # if bifurcation is possible
            bifurcation = max([table[cur_i,k] + table[k+1,cur_j] for k in range(cur_i+1, cur_j)])
        else:
            bifurcation = 0
        
        if T[s[cur_i]] == s[cur_j]: # if we have a match
            match = 1 + table[cur_i+1,cur_j-1]
        else:
            match = 0
            
        down = table[cur_i+1,cur_j]
        left = table[cur_i,cur_j-1]
        
        # tie breaking would happen here
        table[cur_i, cur_j] = max(left, down, match, bifurcation)
        
        cur_i += 1
        cur_j += 1
        
        if cur_j >= len(s):
            cur_i = next_i_start
            cur_j = next_j_start
            next_j_start += 1
    
    if return_table:
        return table
    return table[0, len(s)-1] # top right corner has the answer

In [14]:
s = 'ACGU'
nussinov(s, T)

2.0

In [15]:
s = 'AAAAU'
nussinov(s, T)

1.0

In [16]:
s = 'AAAAUUGC'
nussinov(s, T)

3.0

In [17]:
s = 'GGGAAAUCC'
nussinov(s, T)

3.0

In [18]:
s = 'GCUCGGG UUCCC UAU UCA AGAGC'.replace(' ', '') # should be 10
nussinov(s, T)

10.0